In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
baseline = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\baseline_data.xlsx")

In [ ]:
race_name_dict = baseline['Race Name'].to_dict()

In [ ]:
race_name_mapping = dict(zip(baseline["PCS race entry"], baseline["Race Name"]))

In [ ]:
# Following code pulls GPX coordinates from files

In [ ]:
import os
import gpxpy

gpx_folder = r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\euro classics gpx"

def extract_gpx_coordinates(gpx_file, num_samples=10):
    with open(gpx_file, "r") as f:
        gpx = gpxpy.parse(f)
    
    coordinates = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coordinates.append((point.latitude, point.longitude))
                if len(coordinates) == num_samples:
                    return coordinates  
    return coordinates 

gpx_data = {}

for file in os.listdir(gpx_folder):
    if file.endswith(".gpx"):
        raw_name = file.replace(".gpx", "").strip().lower()  
        formatted_name = raw_name.replace("_", "").replace(" ", "") 
        gpx_path = os.path.join(gpx_folder, file)
        gpx_data[formatted_name] = extract_gpx_coordinates(gpx_path)

gpx_df = pd.DataFrame(list(gpx_data.items()), columns=["PCS race entry", "race_coords"])

baseline = baseline.merge(gpx_df, on='PCS race entry', how='left')

In [ ]:
gpx_df["Race Name"] = gpx_df["PCS race entry"].map(race_name_mapping)

In [ ]:
gpx_df['Year'] = gpx_df['PCS race entry'].str.extract(r'(\d{4})').astype(int)

In [ ]:
gpx_df.loc[98, "Race Name"] = "Ronde van Vlaanderen"

In [ ]:
gpx_df = gpx_df.sort_values(by=['Race Name', 'Year'], ascending=[True, True])

In [ ]:
subset_baseline = baseline[['PCS race entry']]
subset_gpx = gpx_df[['PCS race entry']]

In [ ]:
gpx_df[gpx_df.isnull().any(axis=1)]

In [ ]:
gpx_df.loc[49, "Race Name"] = "Liege - Bastogne - Liege"
gpx_df.loc[68, "Race Name"] = "Ronde van Vlaanderen"

In [ ]:
# After realizing my original pull brought in all the coords for each race, I changed my approach to only include the first, middle, and last sets of coords

In [ ]:
pip install meteostat

In [ ]:
import time
import os
from meteostat import Point, Hourly

In [ ]:
def extract_points(gpx_file):
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)

    points = gpx.tracks[0].segments[0].points
    start = points[0]
    mid = points[len(points) // 2]
    end = points[-1]

    return [
        (start.latitude, start.longitude),
        (mid.latitude, mid.longitude),
        (end.latitude, end.longitude)
    ]

gpx_folder = r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\euro classics gpx"

gpx_files = [os.path.join(gpx_folder, file) for file in os.listdir(gpx_folder) if file.endswith('.gpx')]
total_points = {file: extract_points(file) for file in gpx_files}

In [ ]:
total_points

In [ ]:
import requests

def get_wind_data(lat, lon):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=windspeed_10m,winddirection_10m&timezone=auto"
    
    response = requests.get(url)
    data = response.json()

    wind_speed = data["hourly"]["windspeed_10m"][0] 
    wind_direction = data["hourly"]["winddirection_10m"][0]

    return wind_speed, wind_direction

In [ ]:
results = []

for file, points in total_points.items():
    for i, (lat, lon) in enumerate(points):
        wind_speed, wind_dir = get_wind_data(lat, lon)
        results.append([file, i, lat, lon, wind_speed, wind_dir])

windspeed_df = pd.DataFrame(results, columns=["File", "Index", "Latitude", "Longitude", "Windspeed_kmh", "Wind_direction"])
print(windspeed_df)

windspeed_df.to_csv("wind_speed.csv", index=False)

In [ ]:
windspeed_df['File']

In [ ]:
#windspeed = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\wind_speed.xlsx")

In [ ]:
windspeed_df['Cardinal_direction'] = ''

In [ ]:
# Following code converts wind direction in degrees to compass direction

In [ ]:
def get_cardinal_direction(degrees):
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW', 'N']
    index = round(degrees / 22.5) % 16
    return directions[index]

In [ ]:
def get_cardinal_direction_df(df, wind_col='Wind_direction'):
    df['Cardinal_direction'] = df[wind_col].apply(get_cardinal_direction)

get_cardinal_direction_df(windspeed_df)

In [ ]:
windspeed_df = windspeed_df.drop(columns=['Compass direction', 'Cardinel direction', 'Cardinal direction'])

In [ ]:
windspeed_df.dtypes

In [ ]:
windspeed_df.at[0, 'File']

In [ ]:
windspeed_df['File'] = windspeed_df['File'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

In [ ]:
windspeed_df['File'].unique()

In [ ]:
windspeed_df = windspeed_df.rename(columns={'File': 'Race Name'})

In [ ]:
windspeed_df['Year'] = ''

In [ ]:
windspeed_df['Year'] = windspeed_df['Race Name'].str.extract(r'(\d{4})').astype(int)

In [ ]:
windspeed_df.to_excel('windspeed_df.xlsx')

In [ ]:
windspeed_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\windspeed_df.xlsx")

In [ ]:
windspeed_df = windspeed_df.sort_values(by=['Race Name', 'Year'], ascending=[True, True])

In [ ]:
pps = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\rider_points.xlsx")

In [ ]:
pps = pps.rename(columns={'Rider': 'Winner'})

In [ ]:
baseline = pd.merge(baseline, pps, on='Winner', how='inner')

In [ ]:
for index, column_name in enumerate(baseline.columns):
    print(f"Column index: {index}, Column name: {column_name}")

In [ ]:
baseline = baseline.iloc[:, [0,1,2,3,4,25,26,27,28,29, 5,6,7,8,9, 10, 11,12,13,14,15,16,17,18,19,20,21,22,23]]

In [ ]:
baseline = baseline.rename(columns={'gc': 'GC speciality points', 'time_trial': 'time_trial speciality points', 'sprint': 'sprint speciality points', 'climber': 'climber speciality points', 'hills': 'hills speciality points'})

In [ ]:
baseline.at[0, 'race_coords']

In [ ]:
windspeed_df.at[0, 'File']

In [ ]:
baseline = baseline.drop(columns={'Unnamed: 0', 'race coords'})

In [ ]:
baseline

In [ ]:
baseline.to_excel('baseline_data.xlsx')

In [ ]:
# Race profiles in PCS were hardcoded as html images, so I converted those images into text so that I could use them in PowerBI

In [ ]:
profile_icons = {
    'p1': 'Flat',
    'p2': 'Hills, flat finish',
    'p3': 'Hills, uphill finish',
    'p4': 'Mountains, flat finish',
    'p5': 'Mountains, uphill finish'
}

In [ ]:
baseline['profile_icon'] = baseline['profile_icon'].map(profile_icons)

In [ ]:
baseline.to_excel('baseline_data.xlsx')

In [ ]:
baseline['profile_icon'].value_counts()